In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

In [ ]:


def get_page_links(page):
    results = []
    
    # For paginated pages:
    url = f'https://repository.kallipos.gr/simple-search?query=&filter_field_1=lang&filter_type_1=equals&filter_value_1=el&sort_by=score&order=desc&rpp=100&etal=0&start={page*100 - 100}'
    
    # Static page (for testing):
    # url = 'https://repository.kallipos.gr/handle/11419/14619'
    print(url)

    options = Options()
     
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm -usage")
    service = Service()
    

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 30)
    
    try:
        driver.get(url)

        columns = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'itemListValt2')))

        hrefs = []
        for column in columns:
            links = column.find_elements(By.TAG_NAME, 'a')
            for link in links:
                href = link.get_attribute('href')
                if href:
                    hrefs.append(href)

        

        if len(hrefs) > 100:
            print(f"Found {len(results)} links on page {page}. More than 100 links found.")
        elif len(hrefs) == 0:
            print(f"Fatal error: No links found on page {page}.")
            print('Stopping the program.')
            return False
        elif len(hrefs) < 100:
            print(f"Found {len(results)} links on page {page}. Less than 100 links found.")

        return hrefs

    except Exception as e:
        print(f"An error occurred in get_page_links (selenium) while parsing page {page}: {e}")
        return []
    finally:
        driver.quit()


In [ ]:
get_page_links(1)

In [32]:
from BookScraper import *

url = 'https://repository.kallipos.gr/handle/11419/14417'
scraper =  BookScraper(url = url)
x = scraper.scrape()


In [33]:
x

{'11419_14417': {'links': {'Adobe PDF': '/retrieve/1932e01a-78e5-4d8e-9d27-d2a4c971f8d9/921-MAVROGORDATOS-Field_Theory_Final_TBS_VIII.pdf',
   'Table of Contents - Adobe PDF': '/retrieve/a0571dcf-01a0-4cf8-b0ed-0a6710c58bc4/921-MAVROGORDATOS-Field_Theory_Final_TBS_VΙΙ-TOC.pdf'},
  'metadata': {'Title': 'The Classical Theory of Fields',
   'Abstract': "The Course of Theoretical Physics is a ten-volume series of books that was initiated by Lev Landau and written in collaboration with his student Evgeny Lifshitz, starting in the late 1930s. Since then, the Course has enjoyed worldwide popularity. The hereby translated Volume 2 covers relativistic mechanics of particles, and classical field theory for fields, in particular special relativity and electromagnetism, general relativity and gravitation. It also includes sections on geometrical and wave optics. The fundamental equations are derived from the Principle of Least Action, enabling the attainment of maximum generality together with es

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Base URL of the site
base_url = "https://repository.kallipos.gr/"  # Replace with the actual base domain

# The scraped partial URL
partial_pdf_url = "/retrieve/fe172d8f-26fa-43be-a05c-0fdee2bf61a0/234-PIKOULIS-Manual-of-Surgical-Specialties-vol-ΙΙΙ.pdf"

# Construct full URL
full_url = urljoin(base_url, partial_pdf_url)

# Optional: use session to preserve cookies/headers
session = requests.Session()

# Send POST request to download
response = session.post(full_url)

# Check if the request was successful
if response.status_code == 200:
    with open("Manual-of-Surgical-Specialties-vol-III.pdf", "wb") as f:
        f.write(response.content)
    print("Download complete!")
else:
    print(f"Failed to download PDF. Status code: {response.status_code}")
